In [2]:
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv("./prototype_data.csv")

In [4]:
df.head()

,일시,PM-10,"('풍속', 98)","('풍속', 99)","('풍속', 108)","('풍속', 112)","('풍속', 119)","('풍속', 133)","('풍속', 201)","('강수량', 98)",...,month_11,month_12,hour_00,hour_03,hour_06,hour_09,hour_12,hour_15,hour_18,hour_21
0,2023-01-03 03:00:00,27.0,-1.001528,-1.276354,-0.794339,0.077853,-0.865018,-0.777845,-1.041770,-0.194231,...,False,False,False,True,False,False,False,False,False,False
1,2023-01-03 06:00:00,29.0,-1.001528,-1.178118,-0.692980,-0.654746,-1.128880,-0.777845,-1.209365,-0.194231,...,False,False,False,False,True,False,False,False,False,False
2,2023-01-03 09:00:00,30.0,-0.902350,-1.276354,-0.895697,-1.533866,-1.304788,-0.863775,-0.957973,-0.194231,...,False,False,False,False,False,True,False,False,False,False
3,2023-01-03 12:00:00,41.0,-0.604817,-0.981647,-0.084830,-0.508227,-0.513201,-0.262265,-0.957973,-0.194231,...,False,False,False,False,False,False,True,False,False,False
4,2023-01-03 15:00:00,31.0,0.585315,-0.294000,0.827396,0.297633,0.982019,1.198545,0.466585,-0.194231,...,False,False,False,False,False,False,False,True,False,False


In [5]:
df2 = df.set_index("일시")

In [6]:
df.shape

(2143, 613)

In [7]:
x = df2.drop("PM-10", axis = 1)

In [8]:
y = df2["PM-10"]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state= 42)

In [10]:
# 훈련 세트와 검증 세트로 나누기
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size= 0.3, random_state= 42)

In [11]:
x_train.shape

(1050, 611)

## Seq2seq

In [12]:
# 시퀀스 데이터 생성 함수 (Seq2Seq을 위해 input sequences와 output sequences를 분리)
def create_seq2seq_data(X, y, input_steps=16, output_steps=10): # X: 입력 특징(feature) 데이터프레임. y: 예측 대상 변수 시리즈. 
                                                                # input_steps: 인코더 입력 시퀀스의 길이. output_steps: 디코더 출력 시퀀스의 길이.
    Xs, ys = [], [] # Xs: 인코더 입력 시퀀스 배열. 각 시퀀스는 input_steps 길이의 데이터. ys: 디코더 출력 시퀀스 배열. 각 시퀀스는 output_steps 길이의 데이터.
    for i in range(len(X) - input_steps - output_steps): # X와 y에서 인코더와 디코더에 맞는 시퀀스를 생성
        Xs.append(X.iloc[i:(i + input_steps)].values) # 인코더 입력 시퀀스 생성
        ys.append(y.iloc[(i + input_steps):(i + input_steps + output_steps)].values) # 디코더 출력 시퀀스를 생성
    return np.array(Xs).astype(np.float32), np.array(ys).astype(np.float32)

input_steps = 16
output_steps = 10

# 함수를 사용하여 전체 데이터를 시퀀스 데이터로 변환
X_seq2seq, y_seq2seq = create_seq2seq_data(x, y, input_steps, output_steps) # 인코더 입력 시퀀스 배열. 디코더 출력 시퀀스 배열.

x_train, x_test, y_train, y_test = train_test_split(X_seq2seq, y_seq2seq, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Encoder
encoder_inputs = Input(shape=(input_steps, x_train.shape[2])) # (input_steps, x_train.shape[2])는 각각 시퀀스 길이와 특징(feature) 수를 나타냄
encoder_lstm = LSTM(128, return_state=True) # return_state=True를 사용하여 LSTM의 최종 숨겨진 상태(state_h)와 셀 상태(state_c)를 반환
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c] # 인코더의 상태를 디코더의 초기 상태로 사용하기 위해 리스트에 저장

# Decoder
decoder_inputs = Input(shape=(output_steps, x_train.shape[2])) # (output_steps, x_train.shape[2])는 각각 시퀀스 길이와 특징 수를 나타냄
decoder_lstm = LSTM(128, return_sequences=True, return_state=True) # return_state=True를 사용하여 전체 출력 시퀀스를 반환
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(1, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs) # 디코더의 출력을 1차원 값으로 변환하는 Dense 레이어를 정의

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) # 인코더와 디코더를 연결하여 전체 Seq2Seq 모델을 정의

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = keras.callbacks.ModelCheckpoint("./seq2seq-model.keras", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 8, restore_best_weights = True)
# Model summary
model.summary()

# 디코더 입력 데이터 준비 - 디코더 입력 데이터를 모두 0으로 초기화
# 각 배열의 형태는 (샘플 수, 출력 스텝 수, 특징 수)로 설정
decoder_input_train = np.zeros((y_train.shape[0], output_steps, x_train.shape[2]))
decoder_input_val = np.zeros((y_val.shape[0], output_steps, x_train.shape[2]))
decoder_input_test = np.zeros((y_test.shape[0], output_steps, x_train.shape[2]))

# Training the model
history = model.fit([x_train, decoder_input_train], y_train, epochs=200, validation_data=([x_val, decoder_input_val], y_val))

# Evaluate the model
model.evaluate([x_test, decoder_input_test], y_test)

# Predict
predictions = model.predict([x_test, decoder_input_test])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 16, 611)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 10, 611)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 128), (None,      │         378,880 │ input_layer[0][0]          │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 10, 128), (None,  │         378,880 │ input_layer_1[0][0],       │
│                               │ 128), (None, 128)]        │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 10, 1)             │             129 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 757,889 (2.89 MB)

 Trainable params: 757,889 (2.89 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 1687.9091 - val_loss: 1359.6083
Epoch 2/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1031.0857 - val_loss: 1156.2747
Epoch 3/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 810.3907 - val_loss: 1048.1602
Epoch 4/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 718.1040 - val_loss: 986.3115
Epoch 5/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 649.9003 - val_loss: 957.8717
Epoch 6/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 680.8370 - val_loss: 935.6671
Epoch 7/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 627.1324 - val_loss: 898.9193
Epoch 8/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 594.3788 - val_loss: 843.8958
Epoch 9/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 597.0394 - val_loss: 772.3323
Epoch 10/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 555.1655 - val_loss: 714.5837
Epoch 11/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 475.2685 - val_loss: 676.0203
Epoch 12/200


In [14]:
predictions[:10]

array([[[ 50.833645 ],
        [ 46.22729  ],
        [ 43.92582  ],
        [ 35.685085 ],
        [ 33.75869  ],
        [ 38.49796  ],
        [ 39.813206 ],
        [ 36.77084  ],
        [ 42.234436 ],
        [ 48.631824 ]],

       [[ 16.877317 ],
        [ 21.697895 ],
        [ 20.724983 ],
        [ 18.71252  ],
        [ 18.015448 ],
        [ 27.545742 ],
        [ 27.014322 ],
        [ 24.215242 ],
        [ 29.211443 ],
        [ 27.958649 ]],

       [[ 78.107666 ],
        [ 79.76891  ],
        [ 60.578762 ],
        [ 53.790646 ],
        [ 64.44048  ],
        [ 70.868225 ],
        [ 82.4113   ],
        [ 83.68003  ],
        [ 86.47473  ],
        [101.10961  ]],

       [[ 80.924095 ],
        [ 86.70459  ],
        [ 85.28219  ],
        [ 90.86628  ],
        [ 67.11007  ],
        [ 33.04622  ],
        [ 30.339457 ],
        [ 25.7501   ],
        [ 23.88134  ],
        [ 25.940361 ]],

       [[ 27.138533 ],
        [ 30.169891 ],
        [ 31.846199 ],
   

In [15]:
y_test[:10]

array([[ 48.,  46.,  43.,  40.,  38.,  38.,  39.,  33.,  41.,  40.],
       [ 24.,  15.,  14.,  12.,  24.,  29.,  24.,  31.,  32.,  43.],
       [ 70.,  69.,  77.,  60.,  54.,  67.,  80.,  91.,  76.,  85.],
       [101.,  93.,  81.,  98.,  67.,  35.,  32.,  25.,  21.,  12.],
       [ 29.,  25.,  34.,  27.,  34.,  42.,  54.,  62.,  31.,  25.],
       [ 24.,  46.,  53.,  31.,  27.,  33.,  51.,  74.,  97.,  85.],
       [ 12.,  13.,  39.,  39.,  30.,  28.,  25.,  42.,  61.,  63.],
       [ 20.,  26.,  28.,  27.,  19.,  21.,  23.,  21.,  25.,  20.],
       [ 37.,  38.,  35.,  42.,  49.,  40.,  42.,  49.,  55.,  40.],
       [ 26.,  22.,  15.,  21.,  26.,  21.,  22.,  31.,  26.,  32.]],
      dtype=float32)